In [44]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
dataset = pd.read_csv("/content/drive/MyDrive/Fichiers/2.Scolarité/1. Jedha_Data_Science/CERTIF_PROJECTS/ML_Engineer_Certification_Projects/06_DEEP_LEARNING_At&t/src/src_spam.csv", on_bad_lines='skip', encoding = "cp1252")

### Dataset exploring

In [19]:
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [21]:
dataset["Unnamed: 3"].unique()

array([nan, ' MK17 92H. 450Ppw 16"', ' why to miss them', 'GE',
       'U NO THECD ISV.IMPORTANT TOME 4 2MORO\\""',
       'i wil tolerat.bcs ur my someone..... But',
       ' ILLSPEAK 2 U2MORO WEN IM NOT ASLEEP...\\""',
       'whoever is the KING\\"!... Gud nyt"', ' TX 4 FONIN HON',
       ' \\"OH No! COMPETITION\\". Who knew', 'IåÕL CALL U\\""'],
      dtype=object)

### I will concatenate Unnamed: 2, 3 and 4 because it seams that it is the following of a text message conversation.

In [22]:
columns_to_concatenate = ["v2", "Unnamed: 2", "Unnamed: 3", "Unnamed: 4"]
dataset["text_msg"] = dataset[columns_to_concatenate].astype(str).fillna('').apply(lambda row: ''.join(row), axis=1)
dataset.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,text_msg
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar... Joking wif u oni...nannannan
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,"Nah I don't think he goes to usf, he lives aro..."


In [23]:
dataset = dataset.drop(columns = columns_to_concatenate, axis=1)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   v1        5572 non-null   object
 1   text_msg  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [16]:
# Message len statistiC
len_message = dataset.copy()
len_message["word_count"] = len_message["text_msg"].apply(lambda x : len(x.split()))
mean_len_message = len_message["word_count"].mean()
mean_len_message

15.628320172290021

## Preprocessing the dataset for local training

In [24]:
#encode labels
dataset["target"] = dataset["v1"] == "spam"
dataset.head()

,v1,text_msg,target
0,ham,"Go until jurong point, crazy.. Available only ...",False
1,ham,Ok lar... Joking wif u oni...nannannan,False
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,True
3,ham,U dun say so early hor... U c already then say...,False
4,ham,"Nah I don't think he goes to usf, he lives aro...",False


In [25]:
!python -m spacy download en_core_web_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [26]:
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.lang.en.stop_words import STOP_WORDS

In [29]:
# preprocessing text messages to be able to train the model

dataset["clean_text_msg"] = dataset["text_msg"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
dataset["clean_text_msg"] = dataset["text_msg"].apply(lambda x: x.replace(" +"," ").lower().strip())
dataset["clean_text_msg"] = dataset["text_msg"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))
data_clean = dataset[["clean_text_msg","target"]]
print(data_clean.info())
data_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   clean_text_msg  5572 non-null   object
 1   target          5572 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.1+ KB
None


,clean_text_msg,target
0,"jurong point , crazy .. available bugis n grea...",False
1,ok lar ... joke wif u oni ... nannannan,False
2,free entry 2 wkly comp win FA Cup final tkts 2...,True
3,u dun early hor ... u c ... nannannan,False
4,"nah I think usf , live thoughnannannan",False


In [31]:
# Tokenizing the text messages

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data_clean["clean_text_msg"])
data_clean["text_msg_encoded"] = tokenizer.texts_to_sequences(data_clean["clean_text_msg"])
data_clean["len_msg"] = data_clean["text_msg_encoded"].apply(lambda x: len(x))
data_clean = data_clean[data_clean["len_msg"]!=0]
data_clean.head()

,clean_text_msg,target,text_msg_encoded,len_msg
0,"jurong point , crazy .. available bugis n grea...",False,"[4005, 313, 477, 478, 1072, 31, 53, 199, 1073,...",15
1,ok lar ... joke wif u oni ... nannannan,False,"[12, 226, 677, 314, 3, 1652, 1]",7
2,free entry 2 wkly comp win FA Cup final tkts 2...,True,"[13, 321, 4, 579, 730, 41, 1653, 959, 453, 165...",26
3,u dun early hor ... u c ... nannannan,False,"[3, 132, 181, 2675, 3, 42, 1]",7
4,"nah I think usf , live thoughnannannan",False,"[791, 2, 22, 792, 137, 1076]",6


In [58]:
# Train test split, include target
X_train, X_val, y_train, y_val = train_test_split(data_clean['text_msg_encoded'], data_clean['target'].astype("int"), test_size=0.2, random_state=42)

In [59]:
# Padding data
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding="post")
X_val_pad = tf.keras.preprocessing.sequence.pad_sequences(X_val, padding="post")
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
print(X_train_pad.shape)
print(X_val_pad.shape)
print(y_train.shape)
print(y_val.shape)

(4457, 79)
(1115, 79)
(4457,)
(1115,)


In [63]:
# Create TF datasets

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_pad, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_pad, y_val))

train_dataset = train_dataset.shuffle(len(train_dataset)).batch(64)
val_dataset = val_dataset.batch(64)

### First test : local trained RNN
#### Capture sequences, context of message comprehension for detecting SPAM

In [65]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM, Input, Dropout

vocab_size = len(tokenizer.index_word)+1 #padding
model_1 = tf.keras.Sequential([
                Input([X_train_pad.shape[1],]),
                Embedding(vocab_size, 64, name="embedding"),
                SimpleRNN(units=128, return_sequences=False, go_backwards=True),
                Dropout(0.2),
                Dense(32, activation='relu'),
                Dense(1, activation="sigmoid", name="last")
])

model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 79, 64)            555968    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               24704     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 last (Dense)                (None, 1)                 33        
                                                                 
Total params: 584833 (2.23 MB)
Trainable params: 584833 (2.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
